In [1]:

import pandas as pd, sqlite3

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
conn = sqlite3.connect(':memory:')
df.to_sql('titanic', conn, index=False, if_exists='replace')

# run one SQL example (SELECT first 5 rows)
pd.read_sql_query("SELECT * FROM titanic LIMIT 5;", conn)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [6]:
#List PassengerId, Name, Age, Sex for the first 10 passengers.
query="SELECT PassengerId , Name , Age , Sex FROM titanic LIMIT 10;"
pd.read_sql_query(query,conn)

,PassengerId,Name,Age,Sex
0,1,"Braund, Mr. Owen Harris",22.0,male
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,female
2,3,"Heikkinen, Miss. Laina",26.0,female
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,female
4,5,"Allen, Mr. William Henry",35.0,male
5,6,"Moran, Mr. James",NaN,male
6,7,"McCarthy, Mr. Timothy J",54.0,male
7,8,"Palsson, Master. Gosta Leonard",2.0,male
8,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",27.0,female
9,10,"Nasser, Mrs. Nicholas (Adele Achem)",14.0,female


In [10]:
#Find the top 5 passengers with highest Fare. Show Name, Pclass, Fare.
query2="SELECT Name, Pclass, Fare FROM titanic ORDER BY Fare DESC LIMIT 5;"
pd.read_sql_query(query2,conn)

,Name,Pclass,Fare
0,"Ward, Miss. Anna",1,512.3292
1,"Cardeza, Mr. Thomas Drake Martinez",1,512.3292
2,"Lesurer, Mr. Gustave J",1,512.3292
3,"Fortune, Mr. Charles Alexander",1,263.0000
4,"Fortune, Miss. Mabel Helen",1,263.0000


In [ ]:
#Count how many passengers are older than 60.
query3="SELECT COUNT(*) As 'Number of passenger above the age of 60' FROM titanic WHERE Age > 60 ;"
pd.read_sql_query(query3,conn)

In [22]:
#Compute the average age and average fare by Pclass (display Pclass, avg_age, avg_fare).
query4="SELECT Pclass, AVG(Age) AS avg_age, AVG(Fare) AS avg_fare FROM titanic GROUP BY Pclass;"
pd.read_sql_query(query4,conn)

,Pclass,avg_age,avg_fare
0,1,38.233441,84.154687
1,2,29.877630,20.662183
2,3,25.140620,13.675550


In [23]:
#Find passenger classes with average fare > 40.
query5="SELECT Pclass FROM titanic GROUP BY Pclass HAVING AVG(Fare) > 40;"
pd.read_sql_query(query5,conn)

,Pclass
0,1


In [25]:
#Count passengers per Embarked port (C/Q/S) and show the counts sorted descending.
query6="SELECT Embarked, COUNT(*) AS 'Number of passengers per Embarked Port' FROM titanic GROUP BY Embarked ORDER BY COUNT(*) DESC"
pd.read_sql_query(query6,conn)

,Embarked,Number of passengers per Embarked Port
0,S,644
1,C,168
2,Q,77
3,None,2


In [26]:
#Create a column AgeGroup with values 'Child' (Age < 12), 'Adult' (12 ≤ Age < 60), 'Senior' (Age ≥ 60) and show counts per AgeGroup.
query7="SELECT CASE WHEN Age < 12 THEN 'Child' WHEN Age < 60 THEN 'Adult' ELSE 'Senior' END AS AgeGroup, COUNT(*) AS Count FROM titanic GROUP BY AgeGroup;"
pd.read_sql_query(query7,conn)


,AgeGroup,Count
0,Adult,620
1,Child,68
2,Senior,203


In [29]:
#Find the Name and Fare of passengers who paid a fare higher than the average fare overall.
query8="SELECT Name, Fare FROM titanic WHERE Fare > (SELECT AVG(Fare) FROM titanic)"
pd.read_sql_query(query8,conn)

,Name,Fare
0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",71.2833
1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",53.1000
2,"McCarthy, Mr. Timothy J",51.8625
3,"Sloper, Mr. William Thompson",35.5000
4,"Fortune, Mr. Charles Alexander",263.0000
...,...,...
206,"Wick, Mrs. George Dennick (Mary Hitchcock)",164.8667
207,"Sage, Miss. Dorothy Edith ""Dolly""",69.5500
208,"Roebling, Mr. Washington Augustus II",50.4958
209,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",52.5542


In [30]:
#Show Survived (0/1), COUNT(*) as num_passengers, AVG(Age) as avg_age grouped by Sex and Survived.
query9="SELECT Survived, Sex, COUNT(*) AS num_passengers, AVG(Age) AS avg_age FROM titanic GROUP BY Survived, Sex;"
pd.read_sql_query(query9,conn)

,Survived,Sex,num_passengers,avg_age
0,0,female,81,25.046875
1,0,male,468,31.618056
2,1,female,233,28.847716
3,1,male,109,27.276022


In [31]:
#For survivors only (Survived = 1), find the top 3 Pclass + Sex
#combinations by number of passengers (show Pclass, Sex, count) — useful for joint-group analysis.
query10="SELECT Pclass, Sex, COUNT(*) AS num_passengers FROM titanic WHERE Survived = 1 GROUP BY Pclass, Sex ORDER BY num_passengers DESC LIMIT 3;"
pd.read_sql_query(query10,conn)


,Pclass,Sex,num_passengers
0,1,female,91
1,3,female,72
2,2,female,70
